# Data Exploration

Goals: 
1. Distill the data from American Fact finder to a tidy GeoJSON.

In [ ]:
# !conda install -c conda-forge geopandas -y

In [1]:
import pandas as pd
import geopandas as gpd
import json

/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df = pd.read_csv("../data/interim/ca_educ_by_county_25-55.csv")

In [3]:
df.head()

,countyfips,No schooling completed,"Nursery school, preschool",Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,...,"12th grade, no diploma",Regular high school diploma,GED or alternative credential,"Some college, but less than 1 year","1 or more years of college credit, no degree","Associate's degree, type not specified",Bachelor's degree,Master's degree,Professional degree beyond a bachelor's degree,Doctoral degree
0,0,"5,208",65,-,998,"1,640","4,078","1,607","2,224","16,962",...,"6,173","86,067","10,950","25,048","76,358","37,241","63,418","22,039","6,688","1,731"
1,1,"8,573",-,185,203,294,464,532,819,"5,837",...,"11,954","92,408","9,705","25,099","86,386","38,226","202,890","104,838","21,268","19,029"
2,7,648,-,-,-,-,203,-,-,258,...,"1,511","8,978","1,278","5,638","16,823","5,988","12,985","3,947","1,653",730
3,13,"5,804",-,-,-,549,678,"1,125","1,539","3,696",...,"6,552","49,057","9,237","22,087","60,413","35,853","110,457","50,231","12,138","7,064"
4,17,118,-,-,-,-,-,-,-,-,...,"1,046","5,408","1,292","3,600","9,923","5,663","15,399","4,817","1,755",710


In [4]:
data = df.melt(id_vars='countyfips', var_name = 'education', value_name = 'count')

In [5]:
data['count'] = data['count'].str.strip().str.replace("-","0").str.replace(",","").astype(float)

In [6]:
data.head()

,countyfips,education,count
0,0,No schooling completed,5208.0
1,1,No schooling completed,8573.0
2,7,No schooling completed,648.0
3,13,No schooling completed,5804.0
4,17,No schooling completed,118.0


In [7]:
data['education'].unique()

array(['No schooling completed', 'Nursery school, preschool',
       'Kindergarten', 'Grade 1', 'Grade 2', 'Grade 3', 'Grade 4',
       'Grade 5', 'Grade 6', 'Grade 7', 'Grade 8', 'Grade 9', 'Grade 10',
       'Grade 11', '12th grade, no diploma',
       'Regular high school diploma', 'GED or alternative credential',
       'Some college, but less than 1 year',
       '1 or more years of college credit, no degree',
       "Associate's degree, type not specified", "Bachelor's degree",
       "Master's degree",
       "Professional degree beyond a bachelor's degree",
       'Doctoral degree'], dtype=object)

In [8]:
education_values = [
    'No schooling completed', 
    'Nursery school, preschool',
    'Kindergarten',
    'Grade 1', 'Grade 2', 'Grade 3', 'Grade 4',
    'Grade 5', 'Grade 6', 'Grade 7', 'Grade 8', 
    'Grade 9', 'Grade 10', 'Grade 11', '12th grade, no diploma',
    'Regular high school diploma', 'GED or alternative credential',
    'Some college, but less than 1 year',
    '1 or more years of college credit, no degree',
    "Associate's degree, type not specified", "Bachelor's degree",
    "Master's degree",
    "Professional degree beyond a bachelor's degree",
    'Doctoral degree'
]

In [9]:
data['education'] = pd.Categorical(data['education'], categories = education_values, ordered = True,)

In [10]:
with open('../data/interim/ca_ba.geojson', 'r') as file:
    ca_data = json.load(file)

In [11]:
df = gpd.read_file('../data/interim/county_data/cb_2017_us_county_500k.shp')
df = df[df['STATEFP'] == '06']

In [12]:
data.columns = ['COUNTYFP', 'educd', 'count']

data['COUNTYFP'] = data['COUNTYFP'].astype(str).str.rjust(3, "0")

data.head()

,COUNTYFP,educd,count
0,000,No schooling completed,5208.0
1,001,No schooling completed,8573.0
2,007,No schooling completed,648.0
3,013,No schooling completed,5804.0
4,017,No schooling completed,118.0


In [13]:
df['centroid_lon'] = df['geometry'].centroid.x
df['centroid_lat'] = df['geometry'].centroid.y

df.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,centroid_lon,centroid_lat
18,06,001,01675839,0500000US06001,06001,Alameda,06,1909616630,216916717,"POLYGON ((-122.342253 37.805558, -122.33411840...",-121.888870,37.646895
19,06,005,01675841,0500000US06005,06005,Amador,06,1539933576,29470568,"POLYGON ((-121.027406 38.50354, -121.027472 38...",-120.651090,38.446392
20,06,013,01675903,0500000US06013,06013,Contra Costa,06,1857310903,225193562,"POLYGON ((-122.42976 37.965405, -122.418592 37...",-121.927786,37.919123
21,06,023,01681908,0500000US06023,06023,Humboldt,06,9241251740,1254039383,"POLYGON ((-124.408601 40.44320099999999, -124....",-123.875629,40.699297
22,06,037,00277283,0500000US06037,06037,Los Angeles,06,10510588451,1794793532,"(POLYGON ((-118.604415 33.478552, -118.598783 ...",-118.224817,34.320751


In [14]:
for val in data['educd'].unique():
    print(val)

No schooling completed
Nursery school, preschool
Kindergarten
Grade 1
Grade 2
Grade 3
Grade 4
Grade 5
Grade 6
Grade 7
Grade 8
Grade 9
Grade 10
Grade 11
12th grade, no diploma
Regular high school diploma
GED or alternative credential
Some college, but less than 1 year
1 or more years of college credit, no degree
Associate's degree, type not specified
Bachelor's degree
Master's degree
Professional degree beyond a bachelor's degree
Doctoral degree


In [15]:
data.loc[data['educd'] < 'Regular high school diploma', 'education_level'] = 'No HS'

data.loc[((data['educd'] == 'Regular high school diploma') | (data['educd'] == 'GED or alternative credential')), 'education_level'] = 'HS'

data.loc[(data['educd'] >= 'Some college, but less than 1 year') & (data['educd'] <= "Associate's degree, type not specified"), 'education_level'] = 'Some college/AA'

data.loc[data['educd'] >= "Bachelor's degree", 'education_level'] = 'B.A.+'

#data.loc[data['educd'] > "Bachelor's degree", 'education_level'] = 'Advanced degree'

In [16]:
data['education_level'].isnull().sum()

0

In [17]:
educ_by_county = data.groupby(['COUNTYFP', 'education_level'])[['count']].sum()

shares = educ_by_county.groupby(level=0).apply(lambda x: x / float(x.sum()))

In [18]:
shares.head(25)

count
COUNTYFP education_level          
000      B.A.+            0.236400
         HS               0.244310
         No HS            0.170146
         Some college/AA  0.349144
001      B.A.+            0.534682
         HS               0.156879
         No HS            0.078433
         Some college/AA  0.230006
007      B.A.+            0.308532
         HS               0.163826
         No HS            0.073207
         Some college/AA  0.454435
013      B.A.+            0.453074
         HS               0.146820
         No HS            0.102019
         Some college/AA  0.298086
017      B.A.+            0.433423
         HS               0.128034
         No HS            0.071909
         Some college/AA  0.366635
019      B.A.+            0.231357
         HS               0.206013
         No HS            0.189067
         Some college/AA  0.373563
023      B.A.+            0.372278

In [19]:
percent = shares.groupby(level=0).apply(lambda x: x.iloc[:].max())
educ_level = shares.groupby(level=0).apply(lambda x: x.iloc[:].idxmax())

In [20]:
educ_level = educ_level['count'].str[1].to_frame()

In [21]:
educ_level.columns = ['education_level']
percent.columns = ['share']

In [22]:
data = pd.merge(educ_level, percent, how = 'inner', left_index = True, right_index = True)

In [23]:
data.head()

,education_level,share
COUNTYFP,,
000,Some college/AA,0.349144
001,B.A.+,0.534682
007,Some college/AA,0.454435
013,B.A.+,0.453074
017,B.A.+,0.433423


In [24]:
df = df[['STATEFP', 'COUNTYFP', 'GEOID', 'NAME', 'centroid_lon', 'centroid_lat', 'geometry']]

geo_data = df.merge(data, on  ='COUNTYFP', how = 'left')

geo_data.head()

,STATEFP,COUNTYFP,GEOID,NAME,centroid_lon,centroid_lat,geometry,education_level,share
0,06,001,06001,Alameda,-121.888870,37.646895,"POLYGON ((-122.342253 37.805558, -122.33411840...",B.A.+,0.534682
1,06,005,06005,Amador,-120.651090,38.446392,"POLYGON ((-121.027406 38.50354, -121.027472 38...",NaN,NaN
2,06,013,06013,Contra Costa,-121.927786,37.919123,"POLYGON ((-122.42976 37.965405, -122.418592 37...",B.A.+,0.453074
3,06,023,06023,Humboldt,-123.875629,40.699297,"POLYGON ((-124.408601 40.44320099999999, -124....",B.A.+,0.372278
4,06,037,06037,Los Angeles,-118.224817,34.320751,"(POLYGON ((-118.604415 33.478552, -118.598783 ...",B.A.+,0.370926


In [25]:
geo_data['value_text'] = (geo_data['share'].fillna(0) * 100).astype(str).str.split(".").str[0].str.replace("0", "") + "%"
geo_data.loc[geo_data['value_text'] == '%', 'value_text'] = ""
# geo_data['value_text'] = geo_data['NAME'] + ", " + geo_data['value_text']

In [26]:
geo_data.head()

,STATEFP,COUNTYFP,GEOID,NAME,centroid_lon,centroid_lat,geometry,education_level,share,value_text
0,06,001,06001,Alameda,-121.888870,37.646895,"POLYGON ((-122.342253 37.805558, -122.33411840...",B.A.+,0.534682,53%
1,06,005,06005,Amador,-120.651090,38.446392,"POLYGON ((-121.027406 38.50354, -121.027472 38...",NaN,NaN,
2,06,013,06013,Contra Costa,-121.927786,37.919123,"POLYGON ((-122.42976 37.965405, -122.418592 37...",B.A.+,0.453074,45%
3,06,023,06023,Humboldt,-123.875629,40.699297,"POLYGON ((-124.408601 40.44320099999999, -124....",B.A.+,0.372278,37%
4,06,037,06037,Los Angeles,-118.224817,34.320751,"(POLYGON ((-118.604415 33.478552, -118.598783 ...",B.A.+,0.370926,37%


In [27]:
geo_data['share']=geo_data['share'].fillna(0)
geo_data['education_level']=geo_data['education_level'].fillna("")

In [28]:
geo_data.isnull().sum()

STATEFP            0
COUNTYFP           0
GEOID              0
NAME               0
centroid_lon       0
centroid_lat       0
geometry           0
education_level    0
share              0
value_text         0
dtype: int64

In [29]:
geo_data['name_and_value'] = geo_data['NAME'] + " - " + geo_data['value_text']

geo_data.head()

,STATEFP,COUNTYFP,GEOID,NAME,centroid_lon,centroid_lat,geometry,education_level,share,value_text,name_and_value
0,06,001,06001,Alameda,-121.888870,37.646895,"POLYGON ((-122.342253 37.805558, -122.33411840...",B.A.+,0.534682,53%,Alameda - 53%
1,06,005,06005,Amador,-120.651090,38.446392,"POLYGON ((-121.027406 38.50354, -121.027472 38...",,0.000000,,Amador -
2,06,013,06013,Contra Costa,-121.927786,37.919123,"POLYGON ((-122.42976 37.965405, -122.418592 37...",B.A.+,0.453074,45%,Contra Costa - 45%
3,06,023,06023,Humboldt,-123.875629,40.699297,"POLYGON ((-124.408601 40.44320099999999, -124....",B.A.+,0.372278,37%,Humboldt - 37%
4,06,037,06037,Los Angeles,-118.224817,34.320751,"(POLYGON ((-118.604415 33.478552, -118.598783 ...",B.A.+,0.370926,37%,Los Angeles - 37%


In [30]:
ca_data = json.loads(geo_data.to_json())

with open("../data/interim/ACS_ca_BAs2012-2016.geojson", "w") as file:
    json.dump(ca_data, file, indent=4)

In [31]:
geo_data['education_level'].unique()

array(['B.A.+', '', 'Some college/AA'], dtype=object)

In [32]:
for level in geo_data['education_level'].unique():
    ca_data = json.loads(geo_data[geo_data['education_level']==level].to_json())

    with open(f"../data/interim/ACS_ca_BAs2012-2016__{level.replace('/', '-')}__.geojson", "w") as file:
        json.dump(ca_data, file, indent=4)